# Etiquetado de Tópicos Predefinidos con LLM Multi-Etiqueta

Este notebook implementa un sistema de clasificación multi-etiqueta para reseñas turísticas utilizando un LLM con salida estructurada. El objetivo es etiquetar cada reseña con categorías predefinidas para posteriormente entrenar modelos BERT o similares.

## Import Required Libraries

In [7]:
import pandas as pd
from typing import List
from pydantic import BaseModel, Field, validator
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
import time
import os

## Setup LLM and Pydantic Models

In [8]:
class MultiLabelOutput(BaseModel):
    labels: List[int] = Field(..., description="Lista de números de categorías aplicables (0-14)")
    
    @validator('labels')
    def validate_labels(cls, v):
        valid_labels = set(range(15))
        
        if not all(label in valid_labels for label in v):
            raise ValueError("Todas las etiquetas deben estar entre 0 y 14")
        
        # Manejo automático de la categoría 14 (Otros) cuando aparece con otras etiquetas
        if 14 in v and len(v) > 1:
            # Si la categoría 14 aparece con otras, la removemos automáticamente
            # ya que las otras categorías son más específicas
            v_filtered = [label for label in v if label != 14]
            print(f"⚠️ Categoría 14 removida automáticamente. Etiquetas finales: {v_filtered}")
            return sorted(list(set(v_filtered)))
        
        return sorted(list(set(v)))

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

parser = PydanticOutputParser(pydantic_object=MultiLabelOutput)

/tmp/ipykernel_14278/4131717197.py:4: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @validator('labels')


## Load and Prepare Dataset

In [9]:
df = pd.read_csv('../data/processed/dataset_opiniones_analisis.csv')

print(f"Dataset cargado: {len(df)} reseñas")
print(f"Primeras columnas: {list(df.columns[:5])}")
print(f"Ejemplo de TituloReview: {df['TituloReview'].iloc[0][:100]}...")

Dataset cargado: 2457 reseñas
Primeras columnas: ['Titulo', 'Review', 'Calificacion', 'FechaEstadia', 'Ciudad']
Ejemplo de TituloReview: ¡Divertido y seguro!. Estoy muy impresionado con Mazatlán, Mx. . La gente aquí es amable. . nos aloj...


## Create Classification Prompt

In [10]:
prompt_template = """
Eres un experto en el sector turístico y en etiquetado de datos. Tu tarea es clasificar reseñas turísticas en categorías predefinidas usando etiquetado multi-etiqueta.

CATEGORÍAS DISPONIBLES:
0. Alojamiento - Hoteles, resorts, hospedaje, habitaciones, servicios de hotel
1. Gastronomía - Comida, restaurantes, bebidas, experiencias culinarias
2. Transporte - Medios de transporte, taxis, autobuses, pulmonías, accesibilidad
3. Eventos y festivales - Carnaval, festivales, eventos especiales, espectáculos
4. Historia y cultura - Patrimonio histórico, museos, cultura local, tradiciones
5. Compras - Tiendas, mercados, artesanías, vendedores ambulantes
6. Deportes y aventura - Actividades deportivas, aventura, clavadistas
7. Vida nocturna - Bares, entretenimiento nocturno, ambiente festivo
8. Naturaleza y actividades al aire libre - Paseos, caminatas, actividades en exteriores
9. Playas y actividades acuáticas - Playas, océano, actividades marítimas, natación
10. Personal y servicio - Atención al cliente, amabilidad del personal, servicios
11. Seguridad - Aspectos de seguridad, criminalidad, protección
12. Costo/Precio - Precios, valor por dinero, costos, presupuesto
13. Ambiente/entorno - Ambiente general, atmósfera, limpieza, ruido
14. Otros - Solo usar cuando NINGUNA de las categorías anteriores aplique

REGLAS CRÍTICAS PARA LA CATEGORÍA 14 (Otros):
⚠️ IMPORTANTE: La categoría 14 (Otros) es EXCLUYENTE
⚠️ Si usas [14], NO puedes usar ninguna otra categoría
⚠️ Si hay AL MENOS UNA categoría específica (0-13) que aplique, NO uses la 14
⚠️ La categoría 14 solo se usa cuando absolutamente NINGUNA de las categorías 0-13 es aplicable

EJEMPLOS DE USO CORRECTO:
✅ Correcto: [0, 1, 5] (múltiples categorías específicas)
✅ Correcto: [14] (solo cuando ninguna categoría 0-13 aplica)
❌ INCORRECTO: [10, 14] (nunca combinar 14 con otras)
❌ INCORRECTO: [1, 12, 14] (nunca combinar 14 con otras)

INSTRUCCIONES:
1. Lee cuidadosamente la reseña
2. Identifica TODAS las categorías específicas (0-13) que apliquen
3. Si encontraste AL MENOS UNA categoría específica, usa solo esas (NO incluyas 14)
4. Solo si NO encontraste NINGUNA categoría específica aplicable, entonces usa [14]
5. Devuelve SOLO los números de las categorías como lista

RESEÑA A CLASIFICAR:
{review_text}

{format_instructions}
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["review_text"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

chain = prompt | llm | parser

## Implement Batch Processing Function

In [11]:
def classify_review(review_text, max_retries=2):
    """
    Clasifica una reseña con reintentos automáticos en caso de error.
    """
    for attempt in range(max_retries + 1):
        try:
            result = chain.invoke({"review_text": review_text})
            return result.labels
        except Exception as e:
            error_msg = str(e)
            if attempt < max_retries:
                print(f"⚠️ Intento {attempt + 1} falló, reintentando... Error: {error_msg[:80]}...")
                time.sleep(1)  # Pausa antes del reintento
            else:
                print(f"❌ Error final procesando reseña después de {max_retries + 1} intentos: {error_msg[:100]}...")
                # En caso de error persistente, asignamos categoría "Otros"
                return [14]

def process_reviews_batch(df, start_idx=0, batch_size=50):
    results = []
    total = len(df)
    error_count = 0
    
    for i in range(start_idx, total):
        if i % batch_size == 0:
            print(f"Procesando {i}/{total} reseñas... (Errores hasta ahora: {error_count})")
        
        review = df.iloc[i]['TituloReview']
        labels = classify_review(review)
        
        # Contar si hubo algún error (etiqueta 14 sola puede indicar error)
        if labels == [14]:
            error_count += 1
            
        results.append(labels)
        time.sleep(0.5)
    
    print(f"✅ Procesamiento completado. Total errores/casos 'Otros': {error_count}/{total}")
    return results

## Process Dataset Reviews

In [12]:
print("Clasificando primeras 5 reseñas como prueba...")
test_labels = []
for i in range(5):
    review = df.iloc[i]['TituloReview']
    labels = classify_review(review)
    test_labels.append(labels)
    print(f"Reseña {i+1}: {review[:50]}... -> {labels}")
    time.sleep(1)

print("\nProcesando todo el dataset...")
all_labels = process_reviews_batch(df)

df['TopicoConLLM'] = all_labels

Clasificando primeras 5 reseñas como prueba...
Reseña 1: ¡Divertido y seguro!. Estoy muy impresionado con M... -> [0, 8, 10]
Reseña 1: ¡Divertido y seguro!. Estoy muy impresionado con M... -> [0, 8, 10]
Reseña 2: Un mes en Mazatlán. Acabamos de pasar un mes en Ma... -> [11, 13]
Reseña 2: Un mes en Mazatlán. Acabamos de pasar un mes en Ma... -> [11, 13]
Reseña 3: Maravilloso. Ubicado en la zona más turística de M... -> [1, 5, 6, 13]
Reseña 3: Maravilloso. Ubicado en la zona más turística de M... -> [1, 5, 6, 13]
Reseña 4: Caminando por el Malecón. Hermoso El Malecón de Ma... -> [2, 8]
Reseña 4: Caminando por el Malecón. Hermoso El Malecón de Ma... -> [2, 8]
Reseña 5: Nunca más, por desgracia. Mazatlán no es seguro. L... -> [11]
Reseña 5: Nunca más, por desgracia. Mazatlán no es seguro. L... -> [11]

Procesando todo el dataset...
Procesando 0/2457 reseñas... (Errores hasta ahora: 0)

Procesando todo el dataset...
Procesando 0/2457 reseñas... (Errores hasta ahora: 0)
Procesando 50/2457 re

## Save Results

In [13]:
output_path = '../data/processed/dataset_opiniones_analisis.csv'
df.to_csv(output_path, index=False)

print(f"Dataset actualizado guardado en: {output_path}")
print(f"Total de reseñas procesadas: {len(df)}")

category_names = [
    'Alojamiento', 'Gastronomía', 'Transporte', 'Eventos y festivales', 
    'Historia y cultura', 'Compras', 'Deportes y aventura', 'Vida nocturna',
    'Naturaleza y actividades al aire libre', 'Playas y actividades acuáticas',
    'Personal y servicio', 'Seguridad', 'Costo/Precio', 'Ambiente/entorno', 'Otros'
]

label_counts = {}
for labels_list in df['TopicoConLLM']:
    for label in labels_list:
        label_counts[label] = label_counts.get(label, 0) + 1

print("\nDistribución de etiquetas:")
for i, count in sorted(label_counts.items()):
    category_name = category_names[i]
    print(f"{i}: {category_name} -> {count} reseñas")

multi_label_count = sum(1 for labels in df['TopicoConLLM'] if len(labels) > 1)
print(f"\nReseñas con múltiples etiquetas: {multi_label_count}/{len(df)} ({multi_label_count/len(df)*100:.1f}%)")

sample_multi = df[df['TopicoConLLM'].apply(len) > 1].head(3)
print("\nEjemplos de reseñas multi-etiqueta:")
for idx, row in sample_multi.iterrows():
    print(f"'{row['TituloReview'][:60]}...' -> {row['TopicoConLLM']}")

Dataset actualizado guardado en: ../data/processed/dataset_opiniones_analisis.csv
Total de reseñas procesadas: 2457

Distribución de etiquetas:
0: Alojamiento -> 172 reseñas
1: Gastronomía -> 597 reseñas
2: Transporte -> 207 reseñas
3: Eventos y festivales -> 61 reseñas
4: Historia y cultura -> 847 reseñas
5: Compras -> 348 reseñas
6: Deportes y aventura -> 88 reseñas
7: Vida nocturna -> 55 reseñas
8: Naturaleza y actividades al aire libre -> 562 reseñas
9: Playas y actividades acuáticas -> 430 reseñas
10: Personal y servicio -> 479 reseñas
11: Seguridad -> 96 reseñas
12: Costo/Precio -> 256 reseñas
13: Ambiente/entorno -> 265 reseñas
14: Otros -> 73 reseñas

Reseñas con múltiples etiquetas: 1351/2457 (55.0%)

Ejemplos de reseñas multi-etiqueta:
'¡Divertido y seguro!. Estoy muy impresionado con Mazatlán, M...' -> [0, 8, 10]
'Un mes en Mazatlán. Acabamos de pasar un mes en Mazatlán. No...' -> [11, 13]
'Maravilloso. Ubicado en la zona más turística de Mazatlán, s...' -> [1, 5, 6, 13]
